In [1]:
!nvidia-smi

Sat Jul 29 20:34:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
!mv  kaggle.json /root/.kaggle/kaggle.json

In [13]:
!chmod 600 /root/.kaggle/kaggle.json

In [43]:
!kaggle datasets download -d isaacritharson/severity-based-rice-leaf-diseases-dataset

100% 939M/943M [00:33<00:00, 26.1MB/s]
100% 943M/943M [00:33<00:00, 29.3MB/s]


In [72]:
!unzip  /content/severity-based-rice-leaf-diseases-dataset.zip

Archive:  /content/severity-based-rice-leaf-diseases-dataset.zip
  inflating: Leaf Disease Dataset/train/Healthy/IMG_20190419_095504.jpg  
  inflating: Leaf Disease Dataset/train/Healthy/IMG_20190419_095521.jpg  
  inflating: Leaf Disease Dataset/train/Healthy/IMG_20190419_095611.jpg  
  inflating: Leaf Disease Dataset/train/Healthy/IMG_20190419_095629.jpg  
  inflating: Leaf Disease Dataset/train/Healthy/IMG_20190419_095648.jpg  
  inflating: Leaf Disease Dataset/train/Healthy/IMG_20190419_100056.jpg  
  inflating: Leaf Disease Dataset/train/Healthy/IMG_20190419_100157.jpg  
  inflating: Leaf Disease Dataset/train/Healthy/IMG_20190419_100200.jpg  
  inflating: Leaf Disease Dataset/train/Healthy/IMG_20190419_100204.jpg  
  inflating: Leaf Disease Dataset/train/Healthy/IMG_20190419_100214.jpg  
  inflating: Leaf Disease Dataset/train/Healthy/IMG_20190419_100239.jpg  
  inflating: Leaf Disease Dataset/train/Healthy/IMG_20190419_100418.jpg  
  inflating: Leaf Disease Dataset/train/Healthy

## Install Dependencies

In [19]:
!pip install lightning watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.3 MB/s eta 0:00:00


In [20]:
%load_ext watermark
%watermark -p torch,lightning,torchvision

torch      : 2.0.1+cu118
lightning  : 2.0.6
torchvision: 0.15.2+cu118



## Initialize

In [21]:
import lightning as L
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics
from lightning.pytorch.loggers import CSVLogger
import matplotlib.pyplot as plt
import numpy as np

from shared_utilities import LightningModel,Cifar10DataModule, plot_loss_and_acc

In [22]:
entrypoints = torch.hub.list('pytorch/vision:v0.13.0', force_reload=True)
for e in entrypoints:
    if "resnet" in e:
        print(e)

Downloading: "https://github.com/pytorch/vision/zipball/v0.13.0" to /root/.cache/torch/hub/v0.13.0.zip


deeplabv3_resnet101
deeplabv3_resnet50
fcn_resnet101
fcn_resnet50
resnet101
resnet152
resnet18
resnet34
resnet50
wide_resnet101_2
wide_resnet50_2


In [23]:
pytorch_model = torch.hub.load('pytorch/vision:v0.13.0', 'resnet18', weights='IMAGENET1K_V1')

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.13.0
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 332MB/s]


In [24]:
pytorch_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [25]:
#for param in pytorch_model.parameters():
#    param.requires_grad = False

pytorch_model.fc = torch.nn.Linear(512, 10)

#Custom Transform
Also, we now have to keep in mind the preprocessing protocol that was used for pre-training the model:

In [26]:
from torchvision.models import resnet18, ResNet18_Weights

weights = ResNet18_Weights.IMAGENET1K_V1
preprocess_transform = weights.transforms()
preprocess_transform

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [28]:
%%capture --no-display

L.pytorch.seed_everything(123)

dm = Cifar10DataModule(batch_size=64, num_workers=4,
                       train_transform=preprocess_transform,
                       test_transform=preprocess_transform)

lightning_model = LightningModel(model=pytorch_model, learning_rate=0.1)

trainer = L.Trainer(
    max_epochs=50,
    accelerator="gpu",
    devices=[0],
    logger=CSVLogger(save_dir="logs/", name="my-model"),
    deterministic=True,
)

INFO: Global seed set to 123
INFO:lightning.fabric.utilities.seed:Global seed set to 123
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [29]:
trainer.fit(model=lightning_model, datamodule=dm)

100%|██████████| 170498071/170498071 [00:05<00:00, 29746255.34it/s]


Extracting ./cifar-10-python.tar.gz to ./


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name      | Type               | Params
-------------------------------------------------
0 | model     | ResNet             | 11.2 M
1 | train_acc | MulticlassAccuracy | 0     
2 | val_acc   | MulticlassAccuracy | 0     
3 | test_acc  | MulticlassAccuracy | 0     
-------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name      | Type               | Params
-------------------------------------------------
0 | model     | ResNet             | 11.2 M
1 | train_acc | MulticlassAccuracy | 0     
2 | val_acc   | MulticlassAccuracy | 0     
3 | test_acc  | MulticlassAccuracy | 0     
-------------------------------------------------
11.2 M    Trainable params
0         Non-tra

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [30]:
!ls

 cifar-10-batches-py	  logs	        shared_utilities.py
 cifar-10-python.tar.gz   __pycache__
'Leaf Disease Dataset'	  sample_data


In [31]:
!tar -xvf /content/cifar-10-python.tar.gz

cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1


In [41]:
!tree -L 2 "/content/LeafDiseaseDataset/"

/content/LeafDiseaseDataset/
├── train
│   ├── Healthy
│   ├── Mild Bacterial blight
│   ├── Mild Blast
│   ├── Mild Brownspot
│   ├── Mild Tungro
│   ├── Severe Bacterial blight
│   ├── Severe Blast
│   ├── Severe Brownspot
│   └── Severe Tungro
└── validation
    ├── Healthy
    ├── Mild Bacterial blight
    ├── Mild Blast
    ├── Mild Brownspot
    ├── Mild Tungro
    ├── Severe Bacterial blight
    ├── Severe Blast
    ├── Severe Brownspot
    └── Severe Tungro

20 directories, 0 files


In [53]:
!cp -r /content/LeafDiseaseDataset/validation/* /content/LeafDiseaseDataset/train/

cp: cannot stat '/content/LeafDiseaseDataset/validation/*': No such file or directory


In [58]:
from torchvision import datasets, models, transforms

In [76]:
data_transforms = {
    'train': transforms.Compose([transforms.Resize(299),
        transforms.CenterCrop(299),
#         transforms.RandomResizedCrop(299),
#         transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [77]:
device = "cuda" if torch.cuda.is_available() else "cpu"
val_path="/content/Leaf Disease Dataset/validation"
train_path = "/content/Leaf Disease Dataset/train"

## Classes

In [78]:
from torchvision.datasets import ImageFolder
dataset=ImageFolder(train_path,transform=data_transforms["train"])
img,label=dataset[0]
print(img.shape,label)
print(dataset.classes)

torch.Size([3, 299, 299]) 0
['Healthy', 'Mild Bacterial blight', 'Mild Blast', 'Mild Brownspot', 'Mild Tungro', 'Severe Bacterial blight', 'Severe Blast', 'Severe Brownspot', 'Severe Tungro']


In [79]:
len(dataset)

2826

In [80]:
dataset1=ImageFolder(val_path,transform=data_transforms["val"])
img,label=dataset1[0]
print(img.shape,label)
print(dataset.classes)

torch.Size([3, 299, 299]) 0
['Healthy', 'Mild Bacterial blight', 'Mild Blast', 'Mild Brownspot', 'Mild Tungro', 'Severe Bacterial blight', 'Severe Blast', 'Severe Brownspot', 'Severe Tungro']


In [82]:
len(dataset1)

180

In [83]:
from torch.utils.data import Subset